In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
base_path = ".."
sys.path.append(base_path)
sys.path.append(base_path + "/PropsSimulator")
import numpy as np
import random
import scanpy as sc
import os
import copy


from configs.main_config import config as config_orig
from PropsSimulator.main_config import config as props_config_orig

from prepare_data import dataset
from PropsSimulator.ST_simulator import Simulate, save_dict_to_file

from dissect import run_dissect
from ensemble import ensemble

from src.utils.data_paths_valid import *

import warnings
warnings.filterwarnings('ignore')
# ignore future warning
warnings.simplefilter(action='ignore', category=FutureWarning)

## Define data paths

### Check celltype col

In [3]:
sc_paths, st_paths = get_sc_and_st_paths(base_path)
sc_path = sc_paths[-4]
print(sc_path)
adata = sc.read_h5ad(sc_path)
adata.obs["Experiment"].value_counts()

../data/single-cell/kidney/kidney_full_v3_harmonized_celltypes_raw_mouseGenes.h5ad


Teen Tx    15059
TxK4        5793
RCC1        3522
RCC2        3242
VHL_RCC     3181
Wilms3      2309
RCC3        2239
Wilms1      1927
TxK1        1650
Wilms2       579
TxK3         307
TxK2         160
PapRCC        62
Name: Experiment, dtype: int64

In [4]:
# (adata.obs["celltype_mapped_refined"] == adata.obs["cell_type"]).all()

### Extract relevant paths

In [5]:
sc_paths, st_paths, simulation_paths, experiment_paths = get_paths_for_processing(base_path)

## Check the data

In [6]:
adata = sc.read_h5ad(sc_paths[1])
# adata.obs[["celltype", "broad_celltype", "cell_type"]]
# adata.obs["cell_type"].value_counts()
adata.X.max()
# adata.obs
# check wether celltype_mappped_refined = cell_type
# print(adata.obs["celltype_mapped_refined"].value_counts())
# print(adata.obs["cell_type"].value_counts())

791.0

In [7]:
simulation_paths

['../data/simulated/kidney_slideSeq_v2/UMOD-WT_cell_type',
 '../data/simulated/kidney_slideSeq_v2/UMOD-KI_cell_type',
 '../data/simulated/heart_seqFISH/embryo1_celltype_mapped_refined',
 '../data/simulated/heart_seqFISH/embryo2_celltype_mapped_refined',
 '../data/simulated/heart_seqFISH/embryo3_celltype_mapped_refined',
 '../data/simulated/hypothalamus_MERFISH/0_cell_type',
 '../data/simulated/hypothalamus_MERFISH/1_cell_type',
 '../data/simulated/single-cell/Allenbrain_forSimulation_uniquect_Celltype',
 '../data/simulated/lymph_node/sc_lymph_Subset',
 '../data/simulated/kidney/kidney_full_v3_harmonized_celltypes_raw_mouseGenes_cell_type',
 '../data/simulated/kidney/kidney_full_v3_harmonized_celltypes_raw_mouseGenes_cell_type',
 '../data/simulated/heart_seqFISH/embryo3_celltype_mapped_refined',
 '../data/simulated/heart_seqFISH/embryo3_celltype_mapped_refined',
 '../data/simulated/hypothalamus_MERFISH/2_cell_type',
 '../data/simulated/hypothalamus_MERFISH/2_cell_type',
 '../data/simula

In [8]:
experiment_paths

['../experiments/experiment_kidney_slideSeq_v2/UMOD-WT.WT-2a_resolution75+UMOD-WT_cell_type',
 '../experiments/experiment_kidney_slideSeq_v2/UMOD-KI.KI-4b_resolution105+UMOD-KI_cell_type',
 '../experiments/experiment_heart_seqFISH/embryo1_resolution0.11+embryo1_celltype_mapped_refined',
 '../experiments/experiment_heart_seqFISH/embryo2_resolution0.11+embryo2_celltype_mapped_refined',
 '../experiments/experiment_heart_seqFISH/embryo3_resolution0.11+embryo3_celltype_mapped_refined',
 '../experiments/experiment_hypothalamus_MERFISH/0_resolution0.04+0_cell_type',
 '../experiments/experiment_hypothalamus_MERFISH/1_resolution0.04+1_cell_type',
 '../experiments/experiment_single-cell/V1_Mouse_Brain_Sagittal_Anterior+Allenbrain_forSimulation_uniquect_Celltype',
 '../experiments/experiment_lymph_node/st_lymph+sc_lymph_Subset',
 '../experiments/experiment_kidney/UMOD-WT.WT-2a_resolution75+kidney_full_v3_harmonized_celltypes_raw_mouseGenes_cell_type',
 '../experiments/experiment_kidney/UMOD-KI.KI

In [9]:
spadata = sc.read_h5ad(st_paths[-1])
spadata.X.max()

2516.0

In [10]:
spadata.var_names

Index(['Erg28', '0610009B22Rik', '0610009E02Rik', '0610009L18Rik', 'Dele1',
       '0610010F05Rik', '0610010K14Rik', 'Mettl26', '0610025J13Rik',
       '0610030E20Rik',
       ...
       'mt-Ti', 'mt-Tl1', 'mt-Tl2', 'mt-Tm', 'mt-Tp', 'mt-Tq', 'mt-Tr',
       'mt-Ts2', 'mt-Tt', 'mt-Tv'],
      dtype='object', length=18961)

## Full data pipeline

In [11]:
indices = list(range(len(sc_paths)))[-2::]
indices

[17, 18]

In [12]:
# prepare the config
for idx in indices:
    props_config = copy.deepcopy(props_config_orig)
    props_config["experiment_folder"] = simulation_paths[idx]
    # check if path already exists
    if os.path.exists(props_config["experiment_folder"]):
        print("Data already simulated!")
    else:
        props_config["scdata"] = sc_paths[idx]
        props_config["n_samples"] = None
        if "seqFISH" in sc_paths[idx] or "MERFISH" in sc_paths[idx]:
            props_config["preprocess"] = False
        if "lymph_node" in sc_paths[idx]:
            props_config["n_samples"] = 8000
        props_config["celltype_col"] = celltype_cols[idx]
        if extra_settings_sim[idx] == "per_batch":
            props_config["batch_col"] = "Experiment"
        print(props_config)
        # run simulation
        s = 42
        random.seed(s)
        np.random.seed(s)

        sim = Simulate(props_config)
        if props_config["preprocess"]:
            sim.preprocess()
        sim.generate_props()
        batch_col = sim.config["batch_col"]
        columns = sim.sc_adata.obs.columns
        print(batch_col)
        print(columns)
        if batch_col in columns:
            sim.batches = np.array(sim.sc_adata.obs[sim.config["batch_col"]].unique())
            print(sim.batches)
            if len(sim.batches) > 1:
                sim.simulate_per_batch(save=True)
            else:
                sim.simulate(save=True)
        else:
            print("Number of batches in single-cell data is 1.")
            sim.simulate(save=True)
        save_dict_to_file(props_config)

    config = copy.deepcopy(config_orig)
    config["experiment_folder"] = experiment_paths[idx]
    if os.path.exists(config["experiment_folder"]):
        print("Data already processed!")
    else:
        config["test_dataset"] = st_paths[idx]
        config["reference"] = simulation_paths[idx] + "/simulated.h5ad"
        if "seqFISH" in sc_paths[idx] or "MERFISH" in sc_paths[idx]:
            pass
        else:
            # some optional extra settings
            config["normalize_simulated"] = "cpm"
            config["normalize_test"] = "cpm"
            if extra_settings_prep[idx] == "no_var_cut":
                config["var_cutoff"] = 0.0
            else:
                config["var_cutoff"] = 0.1

        dataset(config)

Data already simulated!
test dataset has 18961 distinct and variable genes.
simulated dataset has 14957 distinct genes.
There are 13507 common genes between simulated and test dataset.
Saving numpy files.
Done.
Data already simulated!
test dataset has 18961 distinct and variable genes.
simulated dataset has 14957 distinct genes.
There are 13507 common genes between simulated and test dataset.
Saving numpy files.
Done.


In [15]:
sim.sc_adata

In [8]:
# check simulated data
sim_adata = sc.read_h5ad(props_config["experiment_folder"] + "/simulated.h5ad")
sim_adata

AnnData object with n_obs × n_vars = 11000 × 23834
    obs: 'Unknown', 'endothelial cell', 'kidney collecting duct intercalated cell', 'kidney collecting duct principal cell', 'kidney distal convoluted tubule epithelial cell', 'kidney interstitial fibroblast', 'kidney loop of Henle thick ascending limb epithelial cell', 'kidney proximal convoluted tubule epithelial cell', 'leukocyte', 'macrophage', 'podocyte'
    obsm: 'cells'
    layers: 'Unknown', 'endothelial cell', 'kidney collecting duct intercalated cell', 'kidney collecting duct principal cell', 'kidney distal convoluted tubule epithelial cell', 'kidney interstitial fibroblast', 'kidney loop of Henle thick ascending limb epithelial cell', 'kidney proximal convoluted tubule epithelial cell', 'leukocyte', 'macrophage', 'podocyte'

## Run DISSECT

In [12]:
indices

[17, 18]

In [13]:
config = copy.deepcopy(config_orig)
for idx in indices:
    config["experiment_folder"] = experiment_paths[idx]
    config["test_dataset"] = st_paths[idx]
    config["reference"] = simulation_paths[idx] + "/simulated.h5ad"
    run_dissect(config)
    ensemble(config)

Loading prepared datasets...
Starting training model 0


2023-04-27 10:39:57.981746: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 10:40:05.619215: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7541 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2023-04-27 10:40:05.620976: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 2086 MB memory:  -> device: 1, name: Tesla V100-SXM2-16GB, pci bus id: 0000:07:00.0, compute capability: 7.0
2023-04-27 10:40:05.622666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:G

Network architecture -
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (64, 512)                 6916096   
                                                                 
 dense_1 (Dense)             (64, 256)                 131328    
                                                                 
 dense_2 (Dense)             (64, 128)                 32896     
                                                                 
 dense_3 (Dense)             (64, 64)                  8256      
                                                                 
 dense_4 (Dense)             (64, 11)                  715       
                                                                 
Total params: 7,089,291
Trainable params: 7,089,291
Non-trainable params: 0
_________________________________________________________________
None


step: 5000| Losses - total_loss: 0.1305 | reg_loss: 0.1254 | cons_loss: 0.0005: 100%|██████████| 5000/5000 [03:34<00:00, 23.31it/s]


2023-04-27 10:43:44.406784: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


...

INFO:tensorflow:Assets written to: ../experiments/experiment_kidney/UMOD-KI.KI-4b_resolution105+kidney_full_v3_harmonized_celltypes_raw_mouseGenes_cell_type+no_var_cut/model_3/assets
Running deconvolution
Estimated proportions are saved at ../experiments/experiment_kidney/UMOD-KI.KI-4b_resolution105+kidney_full_v3_harmonized_celltypes_raw_mouseGenes_cell_type+no_var_cut/dissect_fractions_3.txt.
Starting training model 4


step: 5| Losses - total_loss: 0.8553 | reg_loss: 0.8553 | cons_loss: 0.0000:   0%|          | 4/5000 [00:00<05:26, 15.30it/s]

Network architecture -
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (64, 512)                 6916096   
                                                                 
 dense_46 (Dense)            (64, 256)                 131328    
                                                                 
 dense_47 (Dense)            (64, 128)                 32896     
                                                                 
 dense_48 (Dense)            (64, 64)                  8256      
                                                                 
 dense_49 (Dense)            (64, 11)                  715       
                                                                 
Total params: 7,089,291
Trainable params: 7,089,291
Non-trainable params: 0
_________________________________________________________________
None


step: 5000| Losses - total_loss: 0.1134 | reg_loss: 0.1088 | cons_loss: 0.0005: 100%|██████████| 5000/5000 [03:39<00:00, 22.78it/s]

INFO:tensorflow:Assets written to: ../experiments/experiment_kidney/UMOD-KI.KI-4b_resolution105+kidney_full_v3_harmonized_celltypes_raw_mouseGenes_cell_type+no_var_cut/model_4/assets
Running deconvolution
Estimated proportions are saved at ../experiments/experiment_kidney/UMOD-KI.KI-4b_resolution105+kidney_full_v3_harmonized_celltypes_raw_mouseGenes_cell_type+no_var_cut/dissect_fractions_4.txt.
Ensemble predictions are saved to ../experiments/experiment_kidney/UMOD-KI.KI-4b_resolution105+kidney_full_v3_harmonized_celltypes_raw_mouseGenes_cell_type+no_var_cut/dissect_fractions_ens.txt
